In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import random

In [2]:
H = nx.erdos_renyi_graph(1000,0.6)

In [3]:
nx.is_connected(H)

True

In [4]:
list_of_events = []

In [5]:
times, S, I, R = [0], [len(H.nodes)], [0], [0]

In [6]:
Tmax = 10

In [7]:
beta, gamma = 0.8, 1

In [8]:
for u in H.nodes:
    H.nodes[u]['status'] = 's'
    H.nodes[u]['pred_inf_time'] = np.inf
    #H.nodes[u]['isolation'] = 0

n = 5 #cuántos nodos se aislarán

isolated = np.random.choice(H.nodes, size = n)

for x in isolated:
    H.nodes[x]['isolation'] = 2

In [9]:
u = np.random.choice(H.nodes, size = 1)[0]  #Escoger un nodo para infectar
H.nodes[u]['pred_inf_time'] = 0 #se contagia al inicio

In [10]:
event = (0, u, 'transmit')

In [11]:
list_of_events.append(event)
list_of_events.sort()

In [12]:
def process_recovery(event):
    global H, times, S, I, R
    
    H.nodes[event[1]]['status'] = 'r'
    times.append(event[0])
    S.append(S[-1])
    I.append(I[-1]-1)
    R.append(R[-1]+1)

In [13]:
def process_transmission(event):
    global H, times, S, I, R, list_of_events, beta, gamma, Tmax
    
    if H.nodes[event[1]]['status'] == 's':
        
        H.nodes[event[1]]['status'] = 'i'
        times.append(event[0])
        S.append(S[-1]-1)
        I.append(I[-1]+1)
        R.append(R[-1])
        rec_time = event[0] + random.expovariate(gamma)
        
        if rec_time < Tmax:
            new_event = (rec_time, event[1], 'r')
            list_of_events.append(new_event)
            list_of_events.sort()
            
        for target in H.neighbors(event[1]):
            if H.nodes[target]['status'] == 's':
                inf_time = event[0] + random.expovariate(beta)
                
                if inf_time < min(rec_time, H.nodes[target]['pred_inf_time'], Tmax):
                    new_event = (inf_time, target, 'transmit')
                    list_of_events.append(new_event)
                    list_of_events.sort()
                    H.nodes[target]['pred_inf_time'] = inf_time

In [14]:
while(len(list_of_events) > 0):
    event = list_of_events.pop(0)
    
    if event[2] == 'transmit':
        process_transmission(event)
    else:
        process_recovery(event)

In [15]:
network_data = np.column_stack((times, S, I, R))

In [16]:
pd.DataFrame(network_data).to_csv("network_data.csv")

In [17]:
df = pd.DataFrame(network_data)

In [18]:
df[df[0] < 1]

,0,1,2,3
0,0.000000,1000.0,0.0,0.0
1,0.000000,999.0,1.0,0.0
2,0.000696,998.0,2.0,0.0
3,0.001981,997.0,3.0,0.0
4,0.002038,996.0,4.0,0.0
...,...,...,...,...
1641,0.996097,0.0,359.0,641.0
1642,0.996631,0.0,358.0,642.0
1643,0.997329,0.0,357.0,643.0
1644,0.997669,0.0,356.0,644.0
